In [1]:
#!/usr/bin/env python
# coding: utf-8

# # Берём должников у которых нужно посмотреть

err_not_final = 'в последнем скаченном АСИОИП файле нет "итога"'
dict_log_pas = {"kpi":{'log':'901021350973_210540032049', 'pas':'Kk12345%'}} # kpi

file = 'C:/Users/User/Desktop/py scripts/Список по ограничению на выезд'




from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


import sqlite3
from pathlib import Path
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from tqdm import tqdm
import time
import pandas as pd
from bs4 import BeautifulSoup
import os
import pickle
import math
from selenium.webdriver.support import expected_conditions as EC

from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import date
from datetime import datetime, timedelta
import numpy as np

import sys
import traceback
from selenium.webdriver.support.wait import WebDriverWait
from dateutil.relativedelta import relativedelta

import os
import shutil

from collections import defaultdict
import PyPDF2
from pathlib import Path
import pandas as pd
import re

In [ ]:
def to_site(loading_path_all):
    # Вхожу на сайт
    url = 'https://aisoip.adilet.gov.kz/cabinet/exec-productions'
    
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    
    # ВАЖНО: Настройки для автоматического скачивания
    prefs = {
        "profile.default_content_settings.popups": 0,
        "download.default_directory": loading_path_all.replace('/', '\\'),  # Путь без слеша в конце
        "directory_upgrade": True,
        "download.prompt_for_download": False,  # Не спрашивать где сохранить
        "download.directory_upgrade": True,
        "safebrowsing.enabled": False,  # Отключаем безопасный просмотр (может мешать)
        "profile.default_content_setting_values.automatic_downloads": 1,  # Разрешить автозагрузку
        "profile.content_settings.exceptions.automatic_downloads.*.setting": 1,
    }
    options.add_experimental_option("prefs", prefs)
    
    # Дополнительные опции для обхода блокировок
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    # Подключение к сайту
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    driver.implicitly_wait(10)
    driver.get(url)
    time.sleep(2)
    
    return driver
    
    
def write_login(driver, name_login):
    '''Ввожу логин пароль'''
    login_global   = dict_log_pas[name_login]['log']
    pasword_global = dict_log_pas[name_login]['pas']    
    

    login  = login_global
    pasword = pasword_global

    class_field = 'field'
    field = driver.find_elements(By.CLASS_NAME, class_field)[0]
    field = field.find_element(By.CSS_SELECTOR, 'input')
    field.send_keys(login)

    time.sleep(1)
    field = driver.find_elements(By.CLASS_NAME, class_field)[1]
    field = field.find_element(By.CSS_SELECTOR, 'input')
    field.send_keys(pasword)


    class_field = 'ui primary button fluid'.replace(' ', '.')
    field = driver.find_elements(By.CLASS_NAME, class_field)[0].click()

    time.sleep(1)
    
    
def to_targer_sheet_1(driver):
    # Перехожу на нужную вкладку
    # исполнительные производства
    class_field1 = 'cabinet-header'
    class_field2 = 'navigation-item'
    field = driver.find_element(By.CLASS_NAME, class_field1)
    field = field.find_elements(By.CLASS_NAME, class_field2)[1]
    field.click()
    time.sleep(2)
    
def to_targer_sheet_2(driver):
    # Перехожу на нужную вкладку

    class_field1 = 'v-slide-group__content v-tabs-bar__content'.replace(' ', '.')
    class_field2 = 'div'
    field = driver.find_element(By.CLASS_NAME, class_field1)
    field = field.find_elements(By.CSS_SELECTOR, class_field2)[3]
    field.click()
    time.sleep(1)
    
def iin_insert(driver, iin_iter):
    # Вставляю ИИН
    field = driver.find_element(By.CLASS_NAME, 'row mt-8 mt-md-12'.replace(' ', '.'))
    field = field.find_elements(By.CLASS_NAME, 'v-input__slot'.replace(' ', '.'))[0]
    field = field.find_element(By.CSS_SELECTOR, 'input'.replace(' ', '.'))
    field.send_keys(iin_iter)
    time.sleep(0.1)
    
    
    
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

def buttom_find(driver):
    butom_start_find = '.mb-2 mr-2 mr-md-5 v-btn v-btn--is-elevated v-btn--has-bg theme--light v-size--default primary'.replace(' ', '.')
    field = driver.find_element(By.CSS_SELECTOR, butom_start_find).click()
    time.sleep(3)
    
def close_overlay(driver):
    try:
        overlays = driver.find_elements(By.CSS_SELECTOR, ".v-overlay__scrim")
        for o in overlays:
            driver.execute_script("arguments[0].style.display='none';", o)
        return True
    except:
        return False

def count_data_line(driver):
    # Сохраняю Количество данных "из" 
    class_find = 'v-data-footer__pagination'.replace(' ', '.')
    count_line = driver.find_element(By.CLASS_NAME, class_find).text
    time.sleep(0.5)
    return count_line
    
def find_ispol_on_f258(driver, f258_iter):
    '''Найти подходящий испол. И нажимаем на него.'''
    time.sleep(2)
    
    class_find = 'exec-productions-item mb-2 px-3 px-md-5 py-2 py-md-3 v-card v-sheet theme--light'.replace(' ', '.') #Класс самого объекта строки
    obgect_html = driver.find_elements(By.CLASS_NAME, class_find)
    find_f258_iter = 0
    for row in obgect_html:
        class_find = 'col-sm-6 col-lg-3 col-12'.replace(' ', '.') #Класс одной колонки
        field = row.find_elements(By.CLASS_NAME, class_find)[1] # Выбираю вторую колонку/ячейку
        field = field.find_element(By.CLASS_NAME, 'value')
        text_num_ispol = field.text

        if f258_iter in text_num_ispol:
            field.find_element(By.CSS_SELECTOR, "a[style='text-decoration: underline;']").click()
            time.sleep(1)
            find_f258_iter = 1
            break
    return find_f258_iter


def open_last_production(driver):
    """
    Открывает последнее исполнительное производство
    (считаем, что оно всегда 'На исполнении')
    """
    time.sleep(2)

    class_find = (
        'exec-productions-item mb-2 px-3 px-md-5 py-2 py-md-3 '
        'v-card v-sheet theme--light'
    ).replace(' ', '.')

    productions = driver.find_elements(By.CLASS_NAME, class_find)

    if not productions:
        return False

    last_production = productions[-1]

    link = last_production.find_element(
        By.CSS_SELECTOR,
        "a[style='text-decoration: underline;']"
    )
    link.click()
    time.sleep(2)

    return True


def download_exit_restriction_pdf(driver, download_dir):
    """
    Ищет и скачивает PDF с ограничением на выезд
    """
    time.sleep(2)

    # ищем все ссылки на pdf
    links = driver.find_elements(By.TAG_NAME, 'a')

    for link in links:
        text = link.text.lower()

        if 'огранич' in text and 'выезд' in text:
            link.click()
            return wait_pdf_download(download_dir)

    return None



def wait_pdf_download(download_dir, timeout=30):
    """
    Ждёт появления PDF в папке
    """
    start = time.time()
    download_dir = Path(download_dir)

    while time.time() - start < timeout:
        pdfs = list(download_dir.glob('*.pdf'))
        if pdfs:
            return pdfs[0]
        time.sleep(1)

    return None

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

def find_correct_export_button(driver):
    """
    Находит и анализирует все кнопки экспорта, возвращает правильную
    """
    try:
        # Находим ВСЕ кнопки экспорта на странице
        all_export_buttons = driver.find_elements(
            By.XPATH, 
            "//button[@title='экспорт в Excel']"
        )
        
        print(f"📊 Найдено кнопок экспорта: {len(all_export_buttons)}")
        
        # Анализируем каждую кнопку
        for idx, btn in enumerate(all_export_buttons):
            try:
                # Получаем информацию о родителях
                parent = btn.find_element(By.XPATH, "./ancestor::div[1]")
                grandparent = btn.find_element(By.XPATH, "./ancestor::div[2]")
                
                print(f"\nКнопка #{idx + 1}:")
                print(f"  Родитель: {parent.get_attribute('class')[:50]}...")
                print(f"  Второй родитель: {grandparent.get_attribute('class')[:50]}...")
                
                # Проверяем, находится ли кнопка в блоке документов
                ancestors = driver.execute_script("""
                    var elem = arguments[0];
                    var ancestors = [];
                    while (elem.parentElement) {
                        elem = elem.parentElement;
                        ancestors.push(elem.className);
                    }
                    return ancestors;
                """, btn)
                
                # Ищем блок документов в цепочке родителей
                for i, ancestor_class in enumerate(ancestors):
                    if 'debtor-documents' in ancestor_class:
                        print(f"  ✅ Найдена в блоке документов (уровень {i})")
                        return btn, idx
                
            except Exception as e:
                print(f"  ❌ Ошибка анализа кнопки #{idx}: {e}")
        
        # Если не нашли в документах, возвращаем последнюю кнопку
        if all_export_buttons:
            print(f"⚠️ Не нашли в документах, возвращаем последнюю кнопку")
            return all_export_buttons[-1], len(all_export_buttons) - 1
            
        return None, -1
        
    except Exception as e:
        print(f"❌ Ошибка поиска кнопок: {e}")
        return None, -1
    
    
def click_export_excel_in_documents(driver, timeout=15):
    """
    Кликает на кнопку 'Экспорт в Excel' внутри блока документов
    """
    try:
        print("🔍 Ищем кнопку экспорта внутри блока документов...")
        
        # Ждем загрузки блока документов
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, ".debtor-documents.info-row-item")
            )
        )
        print("✅ Блок документов загружен")
        
        # СПОСОБ 1: Ищем кнопку внутри конкретного блока
        documents_block = driver.find_element(
            By.CSS_SELECTOR, 
            ".debtor-documents.info-row-item"
        )
        
        # Ищем кнопку экспорта ТОЛЬКО внутри этого блока
        export_btn = documents_block.find_element(
            By.XPATH, 
            ".//button[@title='экспорт в Excel']"
        )
        
        # ИЛИ СПОСОБ 2: Более точный селектор
        # export_btn = driver.find_element(
        #     By.CSS_SELECTOR,
        #     ".debtor-documents.info-row-item button[title='экспорт в Excel']"
        # )
        
        print(f"✅ Найдена кнопка внутри блока документов")
        print(f"   Классы: {export_btn.get_attribute('class')}")
        print(f"   Родительский блок: {documents_block.get_attribute('class')}")
        
        # Прокручиваем к блоку
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", documents_block)
        time.sleep(0.5)
        
        # Убираем возможные оверлеи над блоком
        driver.execute_script("""
            // Убираем оверлеи только над этим блоком
            var block = arguments[0];
            var overlays = document.querySelectorAll('.v-overlay__scrim');
            overlays.forEach(function(overlay) {
                var rect1 = block.getBoundingClientRect();
                var rect2 = overlay.getBoundingClientRect();
                
                // Если оверлей перекрывает блок
                if (rect1.top < rect2.bottom && rect1.bottom > rect2.top &&
                    rect1.left < rect2.right && rect1.right > rect2.left) {
                    overlay.style.display = 'none';
                }
            });
        """, documents_block)
        
        # Кликаем с фокусом
        driver.execute_script("""
            arguments[0].focus();
            arguments[0].click();
        """, export_btn)
        
        print("✅ Кнопка экспорта в Excel нажата (внутри блока документов)")
        return True
        
    except Exception as e:
        print(f"❌ Не удалось найти/нажать кнопку внутри блока документов — {e}")
        
        # Попробуем альтернативный способ
        try:
            print("🔄 Пробуем альтернативный поиск...")
            # Ищем ВСЕ кнопки экспорта и берем вторую (первая может быть в другом месте)
            all_export_btns = driver.find_elements(
                By.XPATH, 
                "//button[@title='экспорт в Excel']"
            )
            
            if len(all_export_btns) >= 2:
                print(f"⚠️ Найдено {len(all_export_btns)} кнопок экспорта")
                print("🔼 Пробуем нажать вторую кнопку (скорее всего в документах)")
                
                # Берем вторую кнопку (индекс 1)
                second_btn = all_export_btns[1]
                
                # Проверяем, находится ли она в блоке документов
                parent_classes = second_btn.find_element(By.XPATH, "./ancestor::div[contains(@class, 'debtor-documents')]")
                if parent_classes:
                    print("✅ Вторая кнопка точно в блоке документов")
                
                driver.execute_script("arguments[0].click();", second_btn)
                print("✅ Вторая кнопка экспорта нажата")
                return True
                
        except Exception as e2:
            print(f"❌ Альтернативный способ тоже не сработал — {e2}")
            
        return False
    


def prepare_for_export(driver):
    """Подготавливает данные для экспорта"""
    try:
        # Проверяем, есть ли чекбоксы для выбора данных
        checkboxes = driver.find_elements(By.CSS_SELECTOR, "input[type='checkbox']")
        if checkboxes:
            print(f"Найдено {len(checkboxes)} чекбоксов")
            # Выбираем все чекбоксы
            for cb in checkboxes:
                if not cb.is_selected():
                    driver.execute_script("arguments[0].click();", cb)
                    time.sleep(0.1)
            print("✅ Все чекбоксы отмечены")
            
        # Проверяем, есть ли выбор формата
        format_selectors = driver.find_elements(
            By.XPATH, 
            "//*[contains(text(), 'формат') or contains(text(), 'Format')]"
        )
        if format_selectors:
            print("Найдены элементы выбора формата")
            
    except Exception as e:
        print(f"⚠️ При подготовке к экспорту: {e}")

def close_production_card(driver, timeout=10):
    time.sleep(0.5)
    class_find = '.d-print-none.v-btn.v-btn--absolute.v-btn--icon.v-btn--right.v-btn--round.v-btn--top.theme--light.v-size--small'.replace(' ', '.') #Класс самого объекта строки
    field = driver.find_element(By.CSS_SELECTOR, class_find)
    field.click()


def delite_filter(driver):
    time.sleep(0.3)
    
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.v-btn v-btn--outlined theme--light v-size--default'.replace(' ', '.')))
    )

    field.click()
    


def wait_excel_download(download_dir, timeout=30):
    """
    Ждёт появления Excel-файла после экспорта
    """
    start = time.time()
    download_dir = Path(download_dir)

    while time.time() - start < timeout:
        files = list(download_dir.glob('*.xls*'))
        if files:
            return files[0]
        time.sleep(1)

    return None

import pandas as pd
import os
from datetime import datetime

def extract_dates_from_downloaded_excel(excel_file_path):
    """
    Извлекает ВСЕ даты ограничений / извещений / снятий
    Возвращает:
    ("01.01.2023; 15.03.2023", "20.04.2024")
    """
    try:
        df = pd.read_excel(excel_file_path)
        df.columns = df.columns.str.strip()

        if 'Наименование документа' not in df.columns or 'Дата подписания' not in df.columns:
            return None, None

        restriction_dates = []
        removal_dates = []

        for _, row in df.iterrows():
            doc_name = str(row['Наименование документа']).lower()
            date_str = str(row['Дата подписания'])

            date_obj = None
            for fmt in ['%d.%m.%Y %H:%M:%S', '%d.%m.%Y', '%Y-%m-%d %H:%M:%S']:
                try:
                    date_obj = datetime.strptime(date_str, fmt)
                    break
                except:
                    continue

            if not date_obj:
                continue

            date_formatted = date_obj.strftime('%d.%m.%Y')

            # 🔹 Ограничение + извещение — ВСЕ
            if (
                ('огранич' in doc_name and 'выезд' in doc_name) or
                ('извещ' in doc_name and 'выезд' in doc_name) or 
                ('vremennom ogranichenii' in doc_name)
            ):
                restriction_dates.append(date_formatted)

            # 🔹 Снятие — ВСЕ
            elif 'снят' in doc_name and 'выезд' in doc_name:
                removal_dates.append(date_formatted)

        # ❗ склеиваем ВСЕ даты
        date_restriction = '; '.join(restriction_dates) if restriction_dates else None
        date_removal = '; '.join(removal_dates) if removal_dates else None

        return date_restriction, date_removal

    except Exception as e:
        print(f"❌ Ошибка: {e}")
        return None, None

def update_main_excel_with_dates(main_excel_path, iin, date_restriction, date_removal):
    """
    Обновляет исходный Excel файл с датами ограничения/снятия
    """
    try:
        # Читаем исходный файл
        df_main = pd.read_excel(main_excel_path)
        
        # Проверяем наличие нужных колонок, создаем если их нет
        if 'Постановление об ограничении' not in df_main.columns:
            df_main['Постановление об ограничении'] = None
        if 'Снятие ограничения' not in df_main.columns:
            df_main['Снятие ограничения'] = None
        
        # Находим строку с нужным ИИН
        mask = df_main['ИИН'].astype(str).str.strip() == str(iin).strip()
        
        if mask.any():
            # Обновляем даты
            if date_restriction:
                df_main.loc[mask, 'Постановление об ограничении'] = date_restriction
            if date_removal:
                df_main.loc[mask, 'Снятие ограничения'] = date_removal
            
            # Сохраняем обратно
            df_main.to_excel(main_excel_path, index=False)
            print(f"✅ Файл обновлен для ИИН: {iin}")
            return True
        else:
            print(f"❌ ИИН {iin} не найден в исходном файле")
            return False
            
    except Exception as e:
        print(f"❌ Ошибка обновления файла: {e}")
        return False





def extract_date_from_pdf(pdf_path):
    """
    Достаёт дату постановления из PDF
    """
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        text = ''

        for page in reader.pages:
            text += page.extract_text() or ''

    dates = re.findall(r'\d{2}\.\d{2}\.\d{4}', text)

    if dates:
        return dates[0]   # первая дата = дата постановления

    return None


from pathlib import Path
import pandas as pd

# --- читаем исходный Excel ---
excel_path = r'C:/Users/User/Desktop/py scripts/Список по ограничению на выезд.xlsx'
df = pd.read_excel(excel_path)

# Создаем колонки если их нет
if 'Постановление об ограничении на выезд' not in df.columns:
    df['Постановление об ограничении на выезд'] = None
if 'Снятие об ограничении на выезд' not in df.columns:
    df['Снятие об ограничении на выезд'] = None

# Сохраняем обратно с новыми колонками
df.to_excel(excel_path, index=False)

iin_list = (
    df['ИИН']
    .dropna()
    .astype(str)
    .str.strip()
    .tolist()
)

# Папка для загрузок
download_dir = r"C:/Users/User/Desktop/py scripts/downloads"
Path(download_dir).mkdir(exist_ok=True)

# --- запускаем ---
driver = to_site(download_dir)  # Теперь передаем папку для загрузок
write_login(driver, 'kpi')
to_targer_sheet_1(driver)
to_targer_sheet_2(driver)

# Счетчики для статистики
processed = 0
successful = 0

for iin in iin_list:
    print(f"\n{'='*60}")
    print(f"🔍 Обработка ИИН: {iin}")
    print('='*60)
    
    try:
        # Вводим ИИН и ищем
        iin_insert(driver, iin)
        buttom_find(driver)
        
        # Открываем последнее производство
        if not open_last_production(driver):
            print(f"❌ Не найдено производство для ИИН {iin}")
            continue
        
        print("✅ Карточка производства открыта")
        time.sleep(2)
        
        # Очищаем папку от старых файлов перед скачиванием
        for old_file in Path(download_dir).glob('*.xls*'):
            try:
                old_file.unlink()
                print(f"🗑️ Удален старый файл: {old_file.name}")
            except:
                pass
        
        # Нажимаем кнопку экспорта
        if not click_export_excel_in_documents(driver):
            print("🔄 Пробуем альтернативный способ...")
            # Можно добавить fallback функцию
        
        # Ждем скачивания файла
        excel_file = wait_excel_download(download_dir, timeout=45)
        
        if excel_file:
            print(f"✅ Файл скачан: {excel_file.name}")
            
            # Извлекаем даты из скачанного файла
            date_restriction, date_removal = extract_dates_from_downloaded_excel(excel_file)
            
            # Обновляем исходный файл
            if date_restriction or date_removal:
                update_success = update_main_excel_with_dates(
                    excel_path, iin, date_restriction, date_removal
                )
                if update_success:
                    successful += 1
            else:
                print(f"⚠️ Не найдены даты ограничения/снятия для ИИН {iin}")
        else:
            print(f"❌ Файл не скачан для ИИН {iin}")
        
        # Закрываем карточку
        close_production_card(driver)
        time.sleep(1)
        delite_filter(driver)
        # Удаляем скачанный файл чтобы не мешал
        if excel_file and os.path.exists(excel_file):
            try:
                os.remove(excel_file)
                print(f"🗑️ Удален временный файл: {excel_file.name}")
            except:
                pass
        
        processed += 1
        
    except Exception as e:
        print(f"❌ Ошибка при обработке ИИН {iin}: {e}")
        import traceback
        traceback.print_exc()
    
    # Небольшая пауза между запросами
    time.sleep(1)

print(f"\n{'='*60}")
print(f"📊 ОБРАБОТКА ЗАВЕРШЕНА")
print(f"   Обработано: {processed} из {len(iin_list)}")
print(f"   Успешно обновлено: {successful}")
print('='*60)

# Показываем итоговый результат
df_result = pd.read_excel(excel_path)
print("\n📋 Итоговые данные:")
print(df_result[['ИИН', 'Постановление об ограничении', 'Снятие ограничения']].head(10))


🔍 Обработка ИИН: 811212450106
✅ Карточка производства открыта
🗑️ Удален старый файл: 2103_25-59-469_документы.xlsx
🔍 Ищем кнопку экспорта внутри блока документов...
✅ Блок документов загружен
✅ Найдена кнопка внутри блока документов
   Классы: float-right v-btn v-btn--icon v-btn--round theme--light v-size--default
   Родительский блок: debtor-documents info-row-item
✅ Кнопка экспорта в Excel нажата (внутри блока документов)
✅ Файл скачан: 2103_25-59-469_документы.xlsx
📄 Загружен файл: 2103_25-59-469_документы.xlsx
📊 Колонки: ['Наименование документа', 'Дата подписания']
📈 Количество строк: 12
✅ Найдено ограничение: извещение должника о временном ограничении на выез... - 31.10.2025
📅 Итог: Ограничение=31.10.2025, Снятие=None


c:\Users\User\AppData\Local\Programs\Python\Python314\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


✅ Файл обновлен для ИИН: 811212450106
🗑️ Удален временный файл: 2103_25-59-469_документы.xlsx

🔍 Обработка ИИН: 900103350532
✅ Карточка производства открыта
🔍 Ищем кнопку экспорта внутри блока документов...
✅ Блок документов загружен
✅ Найдена кнопка внутри блока документов
   Классы: float-right v-btn v-btn--icon v-btn--round theme--light v-size--default
   Родительский блок: debtor-documents info-row-item
✅ Кнопка экспорта в Excel нажата (внутри блока документов)
✅ Файл скачан: 2818_25-35-6690_документы.xlsx
📄 Загружен файл: 2818_25-35-6690_документы.xlsx
📊 Колонки: ['Наименование документа', 'Дата подписания']
📈 Количество строк: 7
✅ Найдено ограничение: извещение должника о временном ограничении на выез... - 21.09.2025
📅 Итог: Ограничение=21.09.2025, Снятие=None


c:\Users\User\AppData\Local\Programs\Python\Python314\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


✅ Файл обновлен для ИИН: 900103350532
🗑️ Удален временный файл: 2818_25-35-6690_документы.xlsx

🔍 Обработка ИИН: 860619350058
✅ Карточка производства открыта
🔍 Ищем кнопку экспорта внутри блока документов...
✅ Блок документов загружен
✅ Найдена кнопка внутри блока документов
   Классы: float-right v-btn v-btn--icon v-btn--round theme--light v-size--default
   Родительский блок: debtor-documents info-row-item
✅ Кнопка экспорта в Excel нажата (внутри блока документов)
✅ Файл скачан: 2805_21-19-106_документы.xlsx
📄 Загружен файл: 2805_21-19-106_документы.xlsx
📊 Колонки: ['Наименование документа', 'Дата подписания']
📈 Количество строк: 14
📅 Итог: Ограничение=None, Снятие=None
⚠️ Не найдены даты ограничения/снятия для ИИН 860619350058


...

Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_21992\584825433.py", line 609, in <module>
    iin_insert(driver, iin)
    ~~~~~~~~~~^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_21992\584825433.py", line 88, in iin_insert
    field = driver.find_element(By.CLASS_NAME, 'row mt-8 mt-md-12'.replace(' ', '.'))
  File "c:\Users\User\AppData\Local\Programs\Python\Python314\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 802, in find_element
    return self.execute(Command.FIND_ELEMENT, {"using": by, "value": value})["value"]
           ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\AppData\Local\Programs\Python\Python314\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 432, in execute
    self.error_handler.check_response(response)
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^
  File "c:\Users\User\AppData\Local\Programs\Python\Python314\Lib\site-packages\seleni


🔍 Обработка ИИН: 970411301608
❌ Ошибка при обработке ИИН 970411301608: Message: no such element: Unable to locate element: {"method":"css selector","selector":".row.mt-8.mt-md-12"}
  (Session info: chrome=143.0.7499.170); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7154f88e5
	0x7ff7154f8940
	0x7ff7152d165d
	0x7ff715329a33
	0x7ff715329d3c
	0x7ff71537df67
	0x7ff71537ac97
	0x7ff71531ac29
	0x7ff71531ba93
	0x7ff715810640
	0x7ff71580af80
	0x7ff7158296e6
	0x7ff715515de4
	0x7ff71551ed8c
	0x7ff715502004
	0x7ff7155021b5
	0x7ff7154e7ee2
	0x7fff3c56259d
	0x7fff3dc8af78



Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_21992\584825433.py", line 609, in <module>
    iin_insert(driver, iin)
    ~~~~~~~~~~^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_21992\584825433.py", line 88, in iin_insert
    field = driver.find_element(By.CLASS_NAME, 'row mt-8 mt-md-12'.replace(' ', '.'))
  File "c:\Users\User\AppData\Local\Programs\Python\Python314\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 802, in find_element
    return self.execute(Command.FIND_ELEMENT, {"using": by, "value": value})["value"]
           ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\AppData\Local\Programs\Python\Python314\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 432, in execute
    self.error_handler.check_response(response)
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^
  File "c:\Users\User\AppData\Local\Programs\Python\Python314\Lib\site-packages\seleni


🔍 Обработка ИИН: 940505400949


KeyboardInterrupt: 